In [223]:
import os

import pandas as pd
import numpy as np
import cvxpy as cp

hair(x) → mammal(x)
milk(x) → mammal(x)
feather(x) → bird(x)
layeggs(x) → bird(x)
mammal(x) ∧ meat(x) → carnivore(x)
mammal(x) ∧ pointedteeth(x) ∧ claws(x) ∧ forwardeyes(x) → carnivore(x)
mammal(x) ∧ hoofs(x) → ungulate(x)
mammal(x) ∧ cud(x) → ungulate(x)
carnivore(x) ∧ tawny(x) ∧ darkspots(x) → cheetah(x)
carnivore(x) ∧ tawny(x) ∧ blackstripes(x) → tiger(x)
ungulate(x) ∧ longlegs(x) ∧ longneck(x) ∧ tawny(x) ∧ darkspots(x) → giraffe(x)
ungulate(x) ∧ white(x) ∧ blackstripes(x) → zebra(x)
bird(x) ∧ longlegs(x) ∧ longneck(x) ∧ black(x) → ostrich(x)
bird(x) ∧ swim(x) ∧ blackwhite(x) → penguin(x)
bird(x) ∧ goodflier(x) → albatross(x)
cheetah(x) ⊕ tiger(x) ⊕ giraffe(x) ⊕ zebra(x) ⊕ ostrich(x) ⊕ penguin(x) ⊕ albatross(x)
mammal(x) ⊕ bird(x)
hair(x) ⊕ feather(x)
(darkspots(x)) → ¬ blackstripes(x)
(blackstripes(x)) → ¬ darkspots(x)
tawny(x) → ¬ black(x) ∧¬ white(x)
black(x) → ¬ tawny(x) ∧¬ white(x)
white(x) → ¬ black(x) ∧¬ tawny(x)
black(x) → ¬ white(x)
black(x) → ¬ tawny(x)
white(x) → ¬ black(x)
white(x) → ¬ tawny(x)
tawny(x) → ¬ white(x)
tawny(x) → ¬ black(x)

# Goal
- winston benchmark を解く

## 一時目標 
- toy problem を手動の導出なしで解く
- 


### TO DO
- 演算の定義
- 記号と処理の対応付け
- 演算の順番の実装
- predicate に対しても p_j を割り当て （ex. mammal(x) =: p_2(x）)
- 上から読み込んでいったとしたら 初めてでてきた predicate を認識して


### Notes

- 解釈の順番 否定を先
- 論理式（KB）の書き方を少し制限する
-
- とりあえず，分配法則を使用した書き方を禁止する
- 2重否定もとりあえず許さない
-
- p(x) ○ q(x) 以上の formula を解析しようとするともう少し抽象度をあげなければいけない
- 受け取った formula を空白で区切って要素ごとに分解しリストを作成
- 演算子を認識して前から順に演算
- 演算を終えるごとに，リストを更新し，リストの要素数が 1 かつ，それが演算子でなくなるまで演算を繰り返す
- 
- 
- 与えられた formula の構文チェッカーを作成しても良いかもしれない
- 
- [p_1(x_test), '→', '¬', p_2(x_test), '→', '¬', p_3(x_test)] のような str と cp.Variable が混ざったリストに対して .count や .index ができない
- ループなどを使ってカウントするしか無い
- 
- p を cvxpy で書いたら affine ではなくなった
- おそらく 含意 がわるい


#### 目印になりそうなもの

- 半角スペース
- ()
- () の左に隣接する文字列
- 

# formula の認識，演算の定義，演算子の意味付け

In [39]:
symbols_1 = ['¬', '∧', '∨', '⊗', '⊕', '→']
symbols_2 = ['∀', '∃']

print(f'Symbols: ')
display(symbols_1)
display(symbols_2)
print()

phi_1 = 'hair(x) → mammal(x)'
print(f'Example of phi: ')
display(phi_1)
print()

display(phi_1.split())
print()

phi_2 = 'hair(x) ∧ mammal(x)'
display(phi_2)

Symbols: 


['¬', '∧', '∨', '⊗', '⊕', '→']

['∀', '∃']


Example of phi: 


'hair(x) → mammal(x)'

['hair(x)', '→', 'mammal(x)']

'hair(x) ∧ mammal(x)'

In [40]:
import cvxpy as cp

# ¬ 否定
def negation(x):
    return 1 - x

# ∧ 論理積 and
def weak_conjunction(x, y):
    return cp.minimum(x, y)

# ∨ 論理和 or
def weak_disjunction(x, y):
    return cp.maximum(x, y)

# ⊗ 排他的論理積
def strong_conjunction(x, y):
    return cp.maximum(0, x + y - 1)

# ⊕ 排他的論理和
def strong_disjunction(x, y):
    return cp.minimum(1, x + y)

# → 含意
def implication(x, y):
    return strong_disjunction(negation(x), y)


negation, weak_conjunction, weak_disjunction, strong_conjunction, strong_disjunction, implication

(<function __main__.negation(x)>,
 <function __main__.weak_conjunction(x, y)>,
 <function __main__.weak_disjunction(x, y)>,
 <function __main__.strong_conjunction(x, y)>,
 <function __main__.strong_disjunction(x, y)>,
 <function __main__.implication(x, y)>)

In [42]:
tmp = phi_2.split()

['hair(x)', '∧', 'mammal(x)']

In [48]:
symbols_1 = ['¬', '∧', '∨', '⊗', '⊕', '→']

operations = [
    negation,
    weak_conjunction,
    weak_disjunction,
    strong_conjunction,
    strong_disjunction,
    implication
]

symbols_1_semanticized = {s: o for s, o in zip(symbols_1, operations)}
display(symbols_1_semanticized)

formula = 'hair(x) ∧ mammal(x)'
formula_decomposed = ['hair(x)', '∧', 'mammal(x)']

formula_decomposed_temp = [0.2, '∧', 0.4]

{'¬': <function __main__.negation(x)>,
 '∧': <function __main__.weak_conjunction(x, y)>,
 '∨': <function __main__.weak_disjunction(x, y)>,
 '⊗': <function __main__.strong_conjunction(x, y)>,
 '⊕': <function __main__.strong_disjunction(x, y)>,
 '→': <function __main__.implication(x, y)>}

# 演算子が２つ以上でも計算できるようにする

In [105]:
formula_2 = "mammal(x) ∧ meat(x) → carnivore(x)"
formula_2_decomposed = formula_2.split()


display(formula_2)
display(formula_2_decomposed)

formula_2_decomposed_temp = [0.5, '∧', 0.2, '→', 0.3]
display(formula_2_decomposed_temp)

'mammal(x) ∧ meat(x) → carnivore(x)'

['mammal(x)', '∧', 'meat(x)', '→', 'carnivore(x)']

[0.5, '∧', 0.2, '→', 0.3]

In [106]:
formula_2 = "mammal(x) ∧ meat(x) → carnivore(x)"
formula_2_decomposed = formula_2.split()


display(formula_2)
display(formula_2_decomposed)

formula_2_decomposed_temp = [0.5, '∧', 0.2, '→', 0.3]
display(formula_2_decomposed_temp)


while len(formula_2_decomposed_temp) > 1:
    target_index = None


    # 分割された formula （リスト）から最初の演算子のインデックスを取得
    for index, token in enumerate(formula_2_decomposed_temp):
        if token in symbols_1_semanticized.keys():
            target_index = index
            break

    display(target_index)


    # 対応する演算子の関数を取得
    symbol = formula_2_decomposed_temp[target_index]
    operation = symbols_1_semanticized[symbol]

    display(symbol, operation)

    # 演算に使用する値のペアを取得
    x = formula_2_decomposed_temp[target_index - 1]
    y = formula_2_decomposed_temp[target_index + 1]

    # 実際に演算を行う
    result = operation(x, y)
    display(result.value)

    # 演算結果で置き換え
    formula_2_decomposed_temp[target_index] = result


    indices_to_remove = [target_index - 1, target_index + 1]

    filtered = []

    display(formula_2_decomposed_temp)

    for i, item in enumerate(formula_2_decomposed_temp):
        if i not in indices_to_remove:
            filtered.append(item)


    formula_2_decomposed_temp = filtered

    display(formula_2_decomposed_temp)

1

'∧'

<function __main__.weak_conjunction(x, y)>

array(0.2)

[0.5, Expression(CONSTANT, NONNEGATIVE, ()), 0.2, '→', 0.3]

...

'→'

<function __main__.implication(x, y)>

array(1.)

[Expression(CONSTANT, NONNEGATIVE, ()), Expression(CONSTANT, UNKNOWN, ()), 0.3]

[Expression(CONSTANT, UNKNOWN, ())]

# negation（否定)を先に計算できるようにする

In [146]:
formula_3 = 'darkspots(x) → ¬ blackstripes(x)'
formula_3_decomposed = formula_3.split()
formula_3_decomposed_temp = [0.5, '→', '¬', 0.3, '→', '¬', 0.3, '→', '¬', 0.3]

# display(formula_3_decomposed_temp)




formula_decomposed_tmp = formula_3_decomposed_temp

neg_num = formula_decomposed_tmp.count('¬')

while neg_num > 0:
    target_index = formula_decomposed_tmp.index('¬')

    # 演算に使用する値を取得
    x = formula_decomposed_tmp[target_index + 1]

    # 演算の実行
    operation = symbols_1_semanticized['¬']
    result = operation(x)

    # 演算結果で置き換え，演算子（¬）の削除
    formula_decomposed_tmp[target_index + 1] = result
    formula_decomposed_tmp.pop(target_index)

    neg_num -= 1

    display(formula_decomposed_tmp)


while len(formula_decomposed_tmp) > 1:
    target_index = None

    display(formula_decomposed_tmp)

    # 分割された formula （リスト）から最初の演算子のインデックスを取得
    for index, token in enumerate(formula_decomposed_tmp):
        if token in symbols_1_semanticized.keys():
            target_index = index
            break

    # display(target_index)

    # 対応する演算子の関数を取得
    symbol = formula_decomposed_tmp[target_index]
    operation = symbols_1_semanticized[symbol]

    # display(symbol, operation)

    # 演算に使用する値のペアを取得
    x = formula_decomposed_tmp[target_index - 1]
    y = formula_decomposed_tmp[target_index + 1]

    # 実際に演算を行う
    result = operation(x, y)
    # display(result.value)

    # 演算結果で置き換え，演算済み要素の削除

    formula_decomposed_tmp[target_index] = result
    indices_to_remove = [target_index - 1, target_index + 1]
    filtered = []

    for i, item in enumerate(formula_decomposed_tmp):
        if i not in indices_to_remove:
            filtered.append(item)

    formula_decomposed_tmp = filtered

    # display(formula_decomposed_tmp)


[0.5, '→', 0.7, '→', '¬', 0.3, '→', '¬', 0.3]

[0.5, '→', 0.7, '→', 0.7, '→', '¬', 0.3]

[0.5, '→', 0.7, '→', 0.7, '→', 0.7]

[0.5, '→', 0.7, '→', 0.7, '→', 0.7]

1

[Expression(CONSTANT, NONNEGATIVE, ()), '→', 0.7, '→', 0.7]

1

[Expression(CONSTANT, UNKNOWN, ()), '→', 0.7]

1

# p を cvxpy で書いてみる

In [162]:
len_j = 3 

w_j = cp.Variable(shape=(len_j, 3))

w_1 = w_j[0, :]
w_2 = w_j[1, :]
w_3 = w_j[2, :]


def p_1(x):
    w = w_1
    w1, w2, b = w[0], w[1], w[2]
    x1, x2 = x[0], x[1]
    return w1 * x1 + w2 * x2 + b

def p_2(x):
    w = w_2
    w1, w2, b = w[0], w[1], w[2]
    x1, x2 = x[0], x[1]
    return w1 * x1 + w2 * x2 + b

def p_3(x):
    w = w_3
    w1, w2, b = w[0], w[1], w[2]
    x1, x2 = x[0], x[1]
    return w1 * x1 + w2 * x2 + b

x_test = np.random.rand(2)
x_test

array([0.47049387, 0.88148112])

In [211]:
def count_neg(formula_decomposed):
    neg_num = 0
    
    for item in formula_decomposed:
        if type(item) == str:
            if item == '¬':
                neg_num += 1

    return neg_num

def get_first_neg_index(formula_decomposed):
    target_index = None

    for i, item in enumerate(formula_decomposed):
        if type(item) == str:
            if item == '¬':
                target_index = i
                break
    
    return target_index


In [215]:
formula_3 = 'darkspots(x) → ¬ blackstripes(x)'
formula_3_decomposed = formula_3.split()
formula_3_decomposed_tmp = [0.5, '→', '¬', 0.3, '→', '¬', 0.3, '→', '¬', 0.3]

# display(formula_3_decomposed_temp)

formula_4 = 'p_1(x) → ¬ p_2(x) → ¬ p_3(x)'
formula_4_decomposed = formula_4.split()
formula_4_decomposed_tmp = [p_1(x_test), '→', '¬', p_2(x_test), '→', '¬', p_3(x_test)]

formula_decomposed_tmp = formula_4_decomposed_tmp

display(formula_decomposed_tmp)

neg_num = count_neg(formula_decomposed_tmp)

print(neg_num)

while neg_num > 0:
    target_index = get_first_neg_index(formula_decomposed_tmp)

    # 演算に使用する値を取得
    x = formula_decomposed_tmp[target_index + 1]


    # 演算の実行
    operation = symbols_1_semanticized['¬']
    result = operation(x)

    # 演算結果で置き換え，演算子（¬）の削除
    formula_decomposed_tmp[target_index + 1] = result
    formula_decomposed_tmp.pop(target_index)

    neg_num -= 1

    display(formula_decomposed_tmp)


while len(formula_decomposed_tmp) > 1:
    target_index = None

    display(formula_decomposed_tmp)

    # 分割された formula （リスト）から最初の演算子のインデックスを取得
    for index, token in enumerate(formula_decomposed_tmp):
        if token in symbols_1_semanticized.keys():
            target_index = index
            break

    # display(target_index)

    # 対応する演算子の関数を取得
    symbol = formula_decomposed_tmp[target_index]
    operation = symbols_1_semanticized[symbol]

    # display(symbol, operation)

    # 演算に使用する値のペアを取得
    x = formula_decomposed_tmp[target_index - 1]
    y = formula_decomposed_tmp[target_index + 1]

    # 実際に演算を行う
    result = operation(x, y)
    # display(result.value)

    # 演算結果で置き換え，演算済み要素の削除

    formula_decomposed_tmp[target_index] = result
    indices_to_remove = [target_index - 1, target_index + 1]
    filtered = []

    for i, item in enumerate(formula_decomposed_tmp):
        if i not in indices_to_remove:
            filtered.append(item)

    formula_decomposed_tmp = filtered

    # display(formula_decomposed_tmp)


[Expression(AFFINE, UNKNOWN, ()),
 '→',
 '¬',
 Expression(AFFINE, UNKNOWN, ()),
 '→',
 '¬',
 Expression(AFFINE, UNKNOWN, ())]

2


[Expression(AFFINE, UNKNOWN, ()),
 '→',
 Expression(AFFINE, UNKNOWN, ()),
 '→',
 '¬',
 Expression(AFFINE, UNKNOWN, ())]

[Expression(AFFINE, UNKNOWN, ()),
 '→',
 Expression(AFFINE, UNKNOWN, ()),
 '→',
 Expression(AFFINE, UNKNOWN, ())]

[Expression(AFFINE, UNKNOWN, ()),
 '→',
 Expression(AFFINE, UNKNOWN, ()),
 '→',
 Expression(AFFINE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ()), '→', Expression(AFFINE, UNKNOWN, ())]

In [216]:
formula_decomposed_tmp

[Expression(QUASICONVEX, UNKNOWN, ())]

In [220]:
1 - p_3(x_test)

Expression(AFFINE, UNKNOWN, ())

In [221]:
cp.minimum(1 - p_3(x_test), 0)

Expression(CONCAVE, NONPOSITIVE, ())

# toy モデルの制約を計算してみる

In [231]:
# load and convert data, describe problem settings, etc
data_dir_path = './inputs/toy_data/'
path_L1 = os.path.join(data_dir_path, 'L1.csv')
path_L2 = os.path.join(data_dir_path, 'L2.csv')
path_L3 = os.path.join(data_dir_path, 'L3.csv')
path_U = os.path.join(data_dir_path, 'U.csv')

df_L1 = pd.read_csv(path_L1, index_col=0)
df_L2 = pd.read_csv(path_L2, index_col=0)
df_L3 = pd.read_csv(path_L3, index_col=0)
df_U = pd.read_csv(path_U, index_col=0)

L1 = np.array(df_L1)
L2 = np.array(df_L2)
L3 = np.array(df_L3)

L = np.stack([L1, L2, L3]) # data for pointwise constraint
U = np.array(df_U) # data for logical constriant

In [229]:
len_j = 3 
len_h = 2

w_j = cp.Variable(shape=(len_j, 3))

w_1 = w_j[0, :]
w_2 = w_j[1, :]
w_3 = w_j[2, :]

xi_h = cp.Variable(shape=(len_h, 1), nonneg=True)
xi_1, xi_2 = xi_h[0, :], xi_h[1, :]
xi_1, xi_2


def p_1(x):
    w = w_1
    w1, w2, b = w[0], w[1], w[2]
    x1, x2 = x[0], x[1]
    return w1 * x1 + w2 * x2 + b

def p_2(x):
    w = w_2
    w1, w2, b = w[0], w[1], w[2]
    x1, x2 = x[0], x[1]
    return w1 * x1 + w2 * x2 + b

def p_3(x):
    w = w_3
    w1, w2, b = w[0], w[1], w[2]
    x1, x2 = x[0], x[1]
    return w1 * x1 + w2 * x2 + b

array([0.78173954, 0.96375837])

In [241]:
logical_constraints = []

for u in U:
    logical_constraints += [
        p_1(u) - p_2(u) <= xi_1
    ]

for u in U:
    logical_constraints += [
        p_2(u) - p_3(u) <= xi_2
    ]

logical_constraints

[Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ()))]

In [243]:
u = U[0]
u

array([0.1, 0.5])

In [244]:
formula_decomposed

[Expression(AFFINE, UNKNOWN, ()), '→', Expression(AFFINE, UNKNOWN, ())]

In [252]:
formula_decomposed = [p_1(u), '→', p_2(u)]

display(formula_decomposed)

neg_num = count_neg(formula_decomposed)


while neg_num > 0:
    target_index = get_first_neg_index(formula_decomposed)

    # 演算に使用する値を取得
    x = formula_decomposed[target_index + 1]


    # 演算の実行
    operation = symbols_1_semanticized['¬']
    result = operation(x)

    # 演算結果で置き換え，演算子（¬）の削除
    formula_decomposed[target_index + 1] = result
    formula_decomposed.pop(target_index)

    neg_num -= 1

    display(formula_decomposed)


while len(formula_decomposed) > 1:
    target_index = None

    # 分割された formula （リスト）から最初の演算子のインデックスを取得
    for index, token in enumerate(formula_decomposed):
        if token in symbols_1_semanticized.keys():
            target_index = index
            break

    # display(target_index)

    # 対応する演算子の関数を取得
    symbol = formula_decomposed[target_index]
    operation = symbols_1_semanticized[symbol]

    # display(symbol, operation)

    # 演算に使用する値のペアを取得
    x = formula_decomposed[target_index - 1]
    y = formula_decomposed[target_index + 1]

    # 実際に演算を行う
    result = operation(x, y)
    # display(result.value)

    # 演算結果で置き換え，演算済み要素の削除

    formula_decomposed[target_index] = result
    indices_to_remove = [target_index - 1, target_index + 1]
    filtered = []

    for i, item in enumerate(formula_decomposed):
        if i not in indices_to_remove:
            filtered.append(item)

    formula_decomposed = filtered

    display(formula_decomposed)


[Expression(AFFINE, UNKNOWN, ()), '→', Expression(AFFINE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

In [263]:
def test(formula_decomposed):
    neg_num = count_neg(formula_decomposed)


    while neg_num > 0:
        target_index = get_first_neg_index(formula_decomposed)

        # 演算に使用する値を取得
        x = formula_decomposed[target_index + 1]


        # 演算の実行
        operation = symbols_1_semanticized['¬']
        result = operation(x)

        # 演算結果で置き換え，演算子（¬）の削除
        formula_decomposed[target_index + 1] = result
        formula_decomposed.pop(target_index)

        neg_num -= 1

        display(formula_decomposed)


    while len(formula_decomposed) > 1:
        target_index = None

        # 分割された formula （リスト）から最初の演算子のインデックスを取得
        for index, token in enumerate(formula_decomposed):
            if token in symbols_1_semanticized.keys():
                target_index = index
                break

        # display(target_index)

        # 対応する演算子の関数を取得
        symbol = formula_decomposed[target_index]
        operation = symbols_1_semanticized[symbol]

        # display(symbol, operation)

        # 演算に使用する値のペアを取得
        x = formula_decomposed[target_index - 1]
        y = formula_decomposed[target_index + 1]

        # 実際に演算を行う
        result = operation(x, y)
        # display(result.value)

        # 演算結果で置き換え，演算済み要素の削除

        formula_decomposed[target_index] = result
        indices_to_remove = [target_index - 1, target_index + 1]
        filtered = []

        for i, item in enumerate(formula_decomposed):
            if i not in indices_to_remove:
                filtered.append(item)

        formula_decomposed = filtered

        display(formula_decomposed)

    return formula_decomposed[0]

In [265]:
logical_constraints = []

for u in U:
    formula_decomposed = [p_1(u), '→', p_2(u)]
    formula_calculated = test(formula_decomposed)
    logical_constraints += [
        formula_calculated <= xi_1
    ]

for u in U:
    formula_decomposed = [p_2(u), '→', p_3(u)]
    formula_calculated = test(formula_decomposed)
    logical_constraints += [
        formula_calculated <= xi_2
    ]


[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

...

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

In [266]:
logical_constraints

[Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ())),
 Inequality(Expression(CONCAVE, UNKNOWN, ()))]

In [268]:
# load and convert data, describe problem settings, etc
data_dir_path = './inputs/toy_data/'
path_L1 = os.path.join(data_dir_path, 'L1.csv')
path_L2 = os.path.join(data_dir_path, 'L2.csv')
path_L3 = os.path.join(data_dir_path, 'L3.csv')
path_U = os.path.join(data_dir_path, 'U.csv')

df_L1 = pd.read_csv(path_L1, index_col=0)
df_L2 = pd.read_csv(path_L2, index_col=0)
df_L3 = pd.read_csv(path_L3, index_col=0)
df_U = pd.read_csv(path_U, index_col=0)

L1 = np.array(df_L1)
L2 = np.array(df_L2)
L3 = np.array(df_L3)

L = np.stack([L1, L2, L3]) # data for pointwise constraint
U = np.array(df_U) # data for logical constriant

len_j = 3 # number of tasks (p の数)
len_h = 2 # number of logical constraints considered (cardinality of KB)
len_jl = 0 # number of pointwise constraints to be counted later


len_l_list = [] # L_j の要素数のリスト
len_s_list = [] # S_j の要素数のリスト
S = [] # data for consistency constraints 

for i in range(len_j):
    if len_h != 0:
        u = len(U)
        S_i = np.concatenate((L[i][:, :2], U), axis=0)
        S.append(S_i)
    else:
        u = 0
        S_i = L[i][:, :2]
        S.append(S_i)
    len_l_list.append(len(L[i]))
    len_jl += len(L[i])
    len_s_list.append(len(S_i))

S = np.stack(S)


c1 = 2.5 # degree of satisfaction for pointwise slacks
c2 = 2.5 # degree of satisfaction for logical slacks

c1 = 100
c2 = 100

w_j = cp.Variable(shape=(3 * len_j, 1))  # [w1, w2, b] * 3
xi_j = cp.Variable(shape=(len_jl, 1), nonneg=True) # non-negative とか入れたほうが良さそう
xi_h = cp.Variable(shape=(len_h, 1), nonneg=True)

vars = cp.vstack([w_j, xi_j, xi_h])

# objective_function = cp.Minimize(1/2 * cp.quad_form(vars, H) + f.T @ vars)
func = 0

for j in range(len_j):
    func += 1/2 * cp.norm2(w_j[3*j:3*(j+1)]) ** 2

for jl in range(len_jl):
    func += c1 * xi_j[jl, 0]

for h in range(len_h):
    func += c2 * xi_h[h, 0]

objective_function = cp.Minimize(func)




constraints_2 = []

# pointwise
for j in range(len_j):
    len_l = len_l_list[j]

    w1 = w_j[3 * j + 0, 0]
    w2 = w_j[3 * j + 1, 0]
    b =  w_j[3 * j + 2, 0]

    for l in range(len_l):
        x1 = L[j][l, 0]
        x2 = L[j][l, 1]
        y =  L[j][l, 2]

        xi = xi_j[len_l * j + l, 0]

        constraints_2 += [
            y * (2 * (w1 * x1 + w2 * x2 + b) - 1) >= 1 - 2 * xi
        ]




w1_1 = w_j[3 * 0 + 0, 0]
w2_1 = w_j[3 * 0 + 1, 0]
b_1 =  w_j[3 * 0 + 2, 0]
w1_2 = w_j[3 * 1 + 0, 0]
w2_2 = w_j[3 * 1 + 1, 0]
b_2 =  w_j[3 * 1 + 2, 0]
w1_3 = w_j[3 * 2 + 0, 0]
w2_3 = w_j[3 * 2 + 1, 0]
b_3 =  w_j[3 * 2 + 2, 0]

def p_1(x):
    x1, x2 = x[0], x[1]
    return w1_1 * x1 + w2_1 * x2 + b_1

def p_1(x):
    x1, x2 = x[0], x[1]
    return w1_2 * x1 + w2_2 * x2 + b_2

def p_1(x):
    x1, x2 = x[0], x[1]
    return w1_3 * x1 + w2_3 * x2 + b_3


for u in U:
    formula_decomposed = [p_1(u), '→', p_2(u)]
    formula_calculated = test(formula_decomposed)
    constraints_2 += [
        formula_calculated <= xi_1
    ]

for u in U:
    formula_decomposed = [p_2(u), '→', p_3(u)]
    formula_calculated = test(formula_decomposed)
    constraints_2 += [
        formula_calculated <= xi_2
    ]




# consistency
for j in range(len_j):
    len_s = len_s_list[j]

    w1 = w_j[3 * j + 0, 0]
    w2 = w_j[3 * j + 1, 0]
    b =  w_j[3 * j + 2, 0]

    for s in range(len_s):
        x1 = S[j][s, 0]
        x2 = S[j][s, 1]

        constraints_2 += [
            w1 * x1 + w2 * x2 + b >= 0,
            w1 * x1 + w2 * x2 + b <= 1
        ]


problem = cp.Problem(objective_function, constraints_2)
result = problem.solve(verbose=True)


[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

...

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

[Expression(CONCAVE, UNKNOWN, ())]

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Oct 01 04:48:58 PM: Your problem has 34 variables, 84 constraints, and 0 parameters.
(CVXPY) Oct 01 04:48:58 PM: It is compliant with the following grammars: 
(CVXPY) Oct 01 04:48:58 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 01 04:48:58 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
minimum(1.0, 1.0 + -var2791[6, 0] @ 0.1 + var2791[7, 0] @ 0.5 + var2791[8, 0] + var563[1, 0:3][0] @ 0.1 + var563[1, 0:3][1] @ 0.5 + var563[1, 0:3][2]) <= var567[0, 0] , because the following subexpressions are not:
|--  minimum(1.0, 1.0 + -var2791[6, 0] @ 0.1 + var2791[7, 0] @ 0.5 + var2791[8, 0] + var563[1, 0:3][0] @ 0.1 + var563[1, 0:3][1] @ 0.5 + var563[1, 0:3][2]) <= var567[0, 0]
minimum(1.0, 1.0 + -var2791[6, 0] @ 0.3 + var2791[7, 0] @ 0.7 + var2791[8, 0] + var563[1, 0:3][0] @ 0.3 + var563[1, 0:3][1] @ 0.7 + var563[1, 0:3][2]) <= var567[0, 0] , because the following subexpressions are not:
|--  minimum(1.0, 1.0 + -var2791[6, 0] @ 0.3 + var2791[7, 0] @ 0.7 + var2791[8, 0] + var563[1, 0:3][0] @ 0.3 + var563[1, 0:3][1] @ 0.7 + var563[1, 0:3][2]) <= var567[0, 0]
minimum(1.0, 1.0 + -var2791[6, 0] @ 0.5 + var2791[7, 0] @ 0.4 + var2791[8, 0] + var563[1, 0:3][0] @ 0.5 + var563[1, 0:3][1] @ 0.